# US I94 Immigration Analytics

### Data Engineering Capstone Project

#### Project Summary

This project attempts to build an ETL pipeline hosted on S3 with the use of Spark so as to set up efficient analytics framework with I94 immigration, global land temperatures and US demographics data. Within the project, we will load data from S3, process the data into analytics tables (fact and dimension tables which will act as the fundamental of further business cases analysis) using Spark based on EDA (exploratory data analysis) result, and load them back into S3. 

This project is going to utilize I94 immigration, global land temperatures and US demographics data to build up analytics database. By combining the datasets appropriately, analysis can be done on the dataset to explore the pattern and insight of US immigration along the time.

The data will load in i94 immigration event, land temperature and US demographic dataset, then clean and process the datasets efficiently with the help of module functions and pipeline, then extract the dimension information from cleaned source data and create the dimension tables. With the fact table which store the immigration events and all dimension tables which store the arrive date, visa type, country averag temperature and demographics data, we can do analysis on:
- How international traveler distributes within months over the whole year?
- How international visitor arrival date expand?
- How people choose which port and which travel mode to enter the US? 
- How does age and visa status of international visitor look?
- Is there any relationship between where international visitors come from and the land temperature?

The Spark process will be deployed on a cluster using AWS. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Import libraries

In [15]:
# All imports and installs
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

In [13]:
import pandas as pd
pd.options.display.max_columns = 100
import configparser
import datetime as dt

import utils

# Spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import avg
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.functions import monotonically_increasing_id


In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_spark =spark.read.load('./sas_data')

Exception: Java gateway process exited before sending its port number

In [3]:
df_spark.head()

Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')

In [3]:
# config information
config = configparser.ConfigParser()
config.read('config.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


In [14]:
# create spark session
spark = utils.creat_spark_session()

Exception: Java gateway process exited before sending its port number

### Step 1: Scope the Project and Gather Data

#### Scope 
This project is going to utilize I94 immigration, global land temperatures and US demographics data to build up analytics database. By combining the datasets appropriately, analysis can be done on the dataset to explore the pattern and insight of US immigration along the time.

The first step will be loading the data set with Spark, then appropriate processing and cleansing will be made based on EDA (exploratory data analysis) on I94 immigration, demographics and global land temperatures dataset. Then, dimension and fact tables will be created accordingly so that further advanced analysis can be done on the database.

The project will store the data on Amazon S3 and use Apache Spark to read in source data from staging tables, extract necessary columns needed for analysis and populate the fact and dimension tables. Then will use Spark to write the data back to S3 if needed. For the data modeling part, the project will use the dimensional model which will make it easy for business users to work with the data and also improve analytical queries performance. So, in this case, we will use Star Schema which fits OLAP (online analytical processing) very well.
#### Describe and Gather Data 
The project will be working with four datasets that resides in S3. The main dataset will include data on immigration to the United States, and supplementary datasets will include data on airport codes, U.S. city demographics, and temperature data.

- __I94 Immigration Data:__ This data comes from the US National Tourism and Trade Office. The data contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries).

In [30]:
# Read in the data here

# I94 SAS Data
# fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df_immigration = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
fname = 'immigration_data_sample.csv'
df_immigration = pd.read_csv(fname)
df_immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [7]:
# Read in the data here

# I94 SAS Data
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
df_immigration =spark.read.format('com.github.saurfang.sas.spark').load(fname)


SyntaxError: EOL while scanning string literal (<ipython-input-7-0b42d204b103>, line 4)

In [ ]:
# show first several rows
# df_immigration.limit(5).toPandas()

In [6]:
# write to parquet
# df_immigrationa.write.parquet("sas_data")
# df_immigration=spark.read.parquet("sas_data")

- __World Temperature Data:__ This dataset came from Kaggle. The data contains the information of global average temperature, average temperature uncertainty by country and city.

In [22]:
# read in temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
# df_temperature = spark.read.csv(fname, header=True, inferSchema=True)
df_temperature = pd.read_csv(fname)
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
# display the first five records
# df_temperature.show(n=5)

In [23]:
df_temperature.shape

(8599212, 7)

- __U.S. City Demographic Data:__ This data comes from OpenSoft. This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.

In [24]:
# us cities demoraphics data
fname = 'us-cities-demographics.csv'
df_demographics = pd.read_csv(fname, sep=';')
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [25]:
# display the first five records
# df_demographics.limit(5).toPandas()

In [26]:
df_demographics.shape

(2891, 12)

- __Airport Code Table:__ This is a simple table of airport codes and corresponding cities.

In [27]:
# read in airport codes data
fname = 'airport-codes_csv.csv'
df_airport_codes = pd.read_csv(fname)
df_airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [28]:
df_airport_codes.shape

(55075, 12)

### Step 2: Explore and Assess the Data
#### Explore the Data and Cleaning
Identify data quality issues, like missing values, duplicate data, etc.


- __I94 Immigration Dat__

By looking at the labels mapping of I94 immigration dataset, we can select some candidate columns which will be used in later data modeling. Since we are only looking at 2016 I94 immigration dataset, there’s no need to keep year column, but will keep the month column so that we might be able to get some insight related to month. CIT and Port columns can be kept for exploring the location distribution, mode, birth and visa will be helpful information for analyzing demographic insight. Visapost column can also be useful if we look at where most of international visitor come from, and last will include female and airline columns at first round to see if we can get any valuable information from them.

In [32]:
# Explore the I94 immigration data
df_immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [33]:
# overview on columns
df_immigration.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [104]:
# based on initial analysis, exclude columns which have been decided not to be used
df_immigration_selected = df_immigration[['cicid', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
                                          'i94mode', 'i94addr', 'depdate', 'i94bir', 'visatype',
                                          'count', 'visapost', 'occup', 'gender', 'airline']]

In [105]:
# check the missing values
df_immigration_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
cicid       1000 non-null float64
i94mon      1000 non-null float64
i94cit      1000 non-null float64
i94res      1000 non-null float64
i94port     1000 non-null object
arrdate     1000 non-null float64
i94mode     1000 non-null float64
i94addr     941 non-null object
depdate     951 non-null float64
i94bir      1000 non-null float64
visatype    1000 non-null object
count       1000 non-null float64
visapost    382 non-null object
occup       4 non-null object
gender      859 non-null object
airline     967 non-null object
dtypes: float64(9), object(7)
memory usage: 125.1+ KB


In [106]:
# check missing values
df_immigration_missing = df_immigration_selected.isnull().sum()/df_immigration_selected.shape[0]
df_immigration_missing

cicid       0.000
i94mon      0.000
i94cit      0.000
i94res      0.000
i94port     0.000
arrdate     0.000
i94mode     0.000
i94addr     0.059
depdate     0.049
i94bir      0.000
visatype    0.000
count       0.000
visapost    0.618
occup       0.996
gender      0.141
airline     0.033
dtype: float64

In [107]:
# get columns to drop which have missing values over 50%
drop_cols = list(df_immigration_missing[df_immigration_missing>0.5].index.values)
drop_cols

['visapost', 'occup']

In [108]:
# drop the columns which have missing data > 50% since it won't be helpful for analysis
df_immigration_cols_dropped = df_immigration_selected.drop(drop_cols, axis=1)

In [109]:
# check column names left
df_immigration_cols_dropped.columns

Index(['cicid', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode',
       'i94addr', 'depdate', 'i94bir', 'visatype', 'count', 'gender',
       'airline'],
      dtype='object')

In [110]:
# drop duplicates if exists
df_immigration_dropped = df_immigration_cols_dropped.drop_duplicates()

# drop rows which have missing value in all columns
df_immigration_dropped = df_immigration_dropped.dropna(how='all')

# check shape of processed dataframe
df_immigration_dropped.shape

(1000, 14)

In [111]:
# quick view on processed data
df_immigration_dropped.head()

,cicid,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,visatype,count,gender,airline
0,4084316.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,WT,1.0,F,JL
1,4422636.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,B2,1.0,M,*GA
2,1195600.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,WT,1.0,M,LH
3,5291768.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,B2,1.0,M,QR
4,985523.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,WT,1.0,F,NaN


- __World Temperature data__

In [42]:
# quick look at first several rows
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [43]:
# Overview of the dataset
df_temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [44]:
# check the range of the dt column
df_temperature['dt']=pd.to_datetime(df_temperature['dt'], format="%Y/%m/%d")
display(min(df_temperature['dt']))
display(max(df_temperature['dt']))

Timestamp('1743-11-01 00:00:00')

Timestamp('2013-09-01 00:00:00')

Since the I94 immigration data only has 2016 data, and the temperature has the data till 2013, in that case, we will only look at the 2013 temperature data. And the I94 immigration data only have the information at country level for which country the visitor is from and at most state level for which state the visitor enter the US, in the following steps, we will use this data set at country level.

There’s very little amount of missing data in Average Temperature and Average Temperature Uncertainty columns (4%) and since we will only use temperature dataset at country level, we can just fill the missing data with the group average at country, dt level.

In [45]:
# check missing values
df_temperature_missing = df_temperature.isnull().sum()/df_temperature.shape[0]
df_temperature_missing

dt                               0.000000
AverageTemperature               0.042345
AverageTemperatureUncertainty    0.042345
City                             0.000000
Country                          0.000000
Latitude                         0.000000
Longitude                        0.000000
dtype: float64

In [46]:
# fill missing data with average of the group (country, dt)
df_temperature['AverageTemperature'] = df_temperature.groupby(['Country','dt']).transform(lambda x: x.fillna(x.mean()))
df_temperature['AverageTemperatureUncertainty'] = df_temperature.groupby(['Country','dt']).transform(lambda x: x.fillna(x.mean()))

In [47]:
# recheck missing values
df_temperature.isnull().sum()/df_temperature.shape[0]

dt                               0.000000
AverageTemperature               0.022223
AverageTemperatureUncertainty    0.022223
City                             0.000000
Country                          0.000000
Latitude                         0.000000
Longitude                        0.000000
dtype: float64

In [48]:
# it seems we still have missing data which doesn't have mean in that group, then just remove those records
df_temperature_dropped = df_temperature.dropna(subset=['AverageTemperature','AverageTemperatureUncertainty'])

In [49]:
# check dropped dataframe shape
df_temperature_dropped.shape

(8408116, 7)

In [50]:
# filter only the latest year data and aggregate to country level
df_temperature_2013 = df_temperature_dropped[df_temperature_dropped['dt']>'2012-9-30']
df_temperature_2013.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
3227,2012-10-01,8.320,8.320,Århus,Denmark,57.05N,10.33E
3228,2012-11-01,5.927,5.927,Århus,Denmark,57.05N,10.33E
3229,2012-12-01,-0.921,-0.921,Århus,Denmark,57.05N,10.33E
3230,2013-01-01,-0.344,-0.344,Århus,Denmark,57.05N,10.33E
3231,2013-02-01,-0.390,-0.390,Århus,Denmark,57.05N,10.33E


In [51]:
df_temperature_2013_ctry_month = df_temperature_2013.groupby(['Country','dt'])[['AverageTemperature']].mean().reset_index()
df_temperature_2013_ctry_month.head()

,Country,dt,AverageTemperature
0,Afghanistan,2012-10-01,14.957375
1,Afghanistan,2012-11-01,8.779250
2,Afghanistan,2012-12-01,3.193500
3,Afghanistan,2013-01-01,1.874000
4,Afghanistan,2013-02-01,4.096250


- __U.S. City Demographic Data:__ Looking at US city demographic data, first check the overall missing data and it turned out there’s a vert few missing data (0.5% of the whole dataset at most), so here decide to just remove the rows with missing data. And also remove duplicate rows if exists. 

In [52]:
# quick look at first several rows
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [53]:
# Overview of dataset
df_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [54]:
# check missing data
df_demographics_missing = df_demographics.isnull().sum()/df_demographics.shape[0]
df_demographics_missing

City                      0.000000
State                     0.000000
Median Age                0.000000
Male Population           0.001038
Female Population         0.001038
Total Population          0.000000
Number of Veterans        0.004497
Foreign-born              0.004497
Average Household Size    0.005534
State Code                0.000000
Race                      0.000000
Count                     0.000000
dtype: float64

In [55]:
# it turns that there a very few missing data so that we can just drop the rows where there's missing data
# no need to move any columns
# drop duplicates if exists
df_demographics_dropped = df_demographics.drop_duplicates()

# drop rows which have missing value in columns
df_demographics_dropped = df_demographics_dropped.dropna()

# check shape of processed dataframe
df_demographics_dropped.shape

(2875, 12)

In [56]:
# quick view on processed data
df_demographics_dropped.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


- __Airport Code Data:__ Since after initial assessment, it looks like the airport code dataset is difficult to join the other three datasets especially the I94 immigration data (which doesn’t have city level information). There’s no good option for common column to join the dataset for further analysis, therefore, here decide to not be using this dataset in the data modeling.

In [57]:
# quick look at first several rows
df_airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [58]:
# filter only US airports
df_airport_us = df_airport_codes[df_airport_codes['iso_country']=='US']

In [59]:
# stats on type
df_airport_us.groupby(['type']).size()

type
balloonport          18
closed             1326
heliport           6265
large_airport       170
medium_airport      692
seaplane_base       566
small_airport     13720
dtype: int64

In [60]:
# stats on iso_region
df_airport_us.groupby(['iso_region']).size()

iso_region
US-AK      829
US-AL      361
US-AR      406
US-AZ      359
US-CA     1088
US-CO      505
US-CT      164
US-DC       21
US-DE       57
US-FL      967
US-GA      522
US-HI       64
US-IA      338
US-ID      315
US-IL      902
US-IN      697
US-KS      439
US-KY      257
US-LA      592
US-MA      257
US-MD      257
US-ME      208
US-MI      549
US-MN      569
US-MO      578
US-MS      281
US-MT      331
US-NC      473
US-ND      321
US-NE      309
US-NH      179
US-NJ      442
US-NM      198
US-NV      156
US-NY      668
US-OH      799
US-OK      537
US-OR      492
US-PA      918
US-RI       35
US-SC      217
US-SD      211
US-TN      356
US-TX     2277
US-U-A      10
US-UT      170
US-VA      505
US-VT      102
US-WA      578
US-WI      624
US-WV      140
US-WY      127
dtype: int64

### Step 3: Define the Data Model
In this project, we will use Star Schema to convert the data which is a model designed to support OLAP (online analytics processing) at its best. We will have Fact tables which will record business events (like an order, a phone call, or a book review, etc.) recorded in quantifiable metrics (like quantity of an item, duration of a call, or a book rating, etc.). And Dimension tables will record the context of the business event (e.g., who, what, where, why, etc.) with attributes value like location of a store where the item was purchased, or name of the customer who made the call, etc.
#### 3.1 Conceptual Data Model
The data model will have one fact table that record the key information, and several dimension tables which contains the details. The country dim table has contry mapping and average temperature information. This can support the analysis on how the temperature changing or pattern are related to the immigration trend. The US demographics dimension table grabs information from demographics dataset and connect to the fact table with state level mapping. This will help the analysis on relationship between the immigration pattern and US demographic data. The visa dimension table includes the visa categories information from immigration dataset and can be linked back using visa type key.

#### 3.2 Mapping Out Data Pipelines
The steps to pipeline the data into the chosen data model are as following. First, load in the dataset for processing and cleaning, after getting clean immigration data frame, we can extract visa and arrive date dimension table. After loading the global temperature data, create country dimension table and immigration fact table. Then, load in and clean the demographic data then extract and create the demographic dimension table.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### Immigration date dimension table

In [ ]:
# format the sas date in arrdate column
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)

# create date dimension table using arrdate
df_arrdate_dim = df_immigration_dropped.select(['arrdate']).withColumn("arrdate", get_date(df.arrdate)).distinct()
    
# add other dimension of date
df_arrdate_dim = df_arrdate_dim.withColumn('arrival_day', dayofmonth('arrdate'))
df_arrdate_dim = df_arrdate_dim.withColumn('arrival_week', weekofyear('arrdate'))
df_arrdate_dim = df_arrdate_dim.withColumn('arrival_month', month('arrdate'))
df_arrdate_dim = df_arrdate_dim.withColumn('arrival_year', year('arrdate'))
df_arrdate_dim = df_arrdate_dim.withColumn('arrival_weekday', dayofweek('arrdate'))

# add an identical column
df_arrdate_dim = df_arrdate_dim.withColumn('id', monotonically_increasing_id())
    
# # write to parquet file
# partition_columns = ['arrival_year', 'arrival_month', 'arrival_week']
# df_arrdate_dim.write.parquet("immigration_tables/" + "immigration_arrdate", partitionBy=partition_columns, mode="overwrite")


#### Country dimension table

In [62]:
# country codes mapping
country_cd = pd.read_csv('i94res.csv')
country_cd.head()

,Code,Name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [112]:
# aggregate the temperature data
agg_temp = df_temperature_dropped.groupby('Country')[['AverageTemperature']].mean().reset_index()
agg_temp['Country'] = agg_temp['Country'].str.upper()
agg_temp.head()

,Country,AverageTemperature
0,AFGHANISTAN,13.764405
1,ALBANIA,15.525828
2,ALGERIA,17.763206
3,ANGOLA,21.759161
4,ARGENTINA,16.999216


In [113]:
# extract the i94res column
df_country_dim = df_immigration_dropped[['i94res']].drop_duplicates()
df_country_dim.columns = ['country_code']
df_country_dim.head()

,country_code
0,209.0
1,582.0
2,112.0
3,297.0
4,111.0


In [114]:
# add country name
df_country_dim['country_name']=df_country_dim['country_code'].apply(lambda x: country_cd[country_cd['Code']==x]['Name'].iloc[0])
df_country_dim.head()

,country_code,country_name
0,209.0,JAPAN
1,582.0,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
2,112.0,GERMANY
3,297.0,QATAR
4,111.0,FRANCE


In [103]:
# add country average temperature
df_country_dim['average_temperature']=df_country_dim['country_name'].apply(lambda x: None if agg_temp[agg_temp['Country']==x]['AverageTemperature'].empty \
                                                                         else agg_temp[agg_temp['Country']==x]['AverageTemperature'].iloc[0])
df_country_dim.head()

,country_code,country_name,average_temperature
0,209.0,JAPAN,13.394260
1,582.0,"MEXICO Air Sea, and Not Reported (I-94, no lan...",NaN
2,112.0,GERMANY,8.482791
3,297.0,QATAR,26.831705
4,111.0,FRANCE,10.337918


#### Visa dimension table

In [ ]:
# create visa dimension dataframe using visatype column
df_visa_dim = df_immigration_dropped.select(['visatype']).distinct()

# add an nonduplicate id column
df_visa_dim = df_visa_dim.withColumn('visa_type_key', monotonically_increasing_id())


#### Demographic dimension table

In [ ]:
df_demographics_dim = df_demographics_dropped.withColumnRenamed('Median Age', 'median_age') \
    .withColumnRenamed('Male Population', 'male_population') \
    .withColumnRenamed('Female Population', 'female_population') \
    .withColumnRenamed('Total Population', 'total_population') \
    .withColumnRenamed('Number of Veterans', 'number_of_veterans') \
    .withColumnRenamed('Foreign-born', 'foreign_born') \
    .withColumnRenamed('Average Household Size', 'average_household_size') \
    .withColumnRenamed('State Code', 'state_code')

# add an un duplicate id column
df_demographics_dim = df_demographics_dim.withColumn('id', monotonically_increasing_id())


#### Immigration fact table

In [ ]:
# create a view for visa type dimension
df_visa.createOrReplaceTempView("visa_view")

# map the visa type code
@udf('string')
def add_visa_key(visa_type):
    """map visa key
    
    Args:
    visa_type: US non-immigrant visa type
    
    Returns:
        corresponding visa key
    """
    key_series = df_visa[df_visa['visatype']==visa_type]['visa_type_key']
    
    if not key_series.empty:
        return str(key_series.iloc[0])
    
    return None
    
# convert arrival date in SAS format to datetime
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)

# rename columns to align with data model
df_fact = df_immigration_dropped.withColumnRenamed('ccid', 'record_id') \
    .withColumnRenamed('i94res', 'country_residence_code') \
    .withColumnRenamed('i94addr', 'state_code')

# add visa_type key
df_fact = df_fact.withColumn('visa_type_key', add_visa_key('visatype'))
    
# format arrival date into datetime object
df_fact = df_fact.withColumn("arrdate", get_date(df_fact.arrdate))


#### 4.2 Data Quality Checks
The data quality checks to ensure the pipeline ran as expected.

In [ ]:
# Perform quality checks

tables = {
    'immigration_fact': df_fact,
    'visa_dim': df_visa_dim,
    'arrdate_dim': df_arrdate_dim,
    'demographics_dim': df_demographics_dim,
    'country_dim': df_country_dim
}

# check if the table is loaded without error
for table_name, df_table in tables.items():
    # quality check for table
    total_count = input_data.count()

    if total_count == 0:
        print(f"Data quality check failed for {table_name} with zero records!")
    else:
        print(f"Data quality check passed for {table_name} with {total_count:,} records.")


In [ ]:
# check if the table counts is same as the source table

# quality check for table
df_immigration_count = df_immigration_dropped.count()
df_fact_count = df_fact.count()

if df_immigration_count == df_fact_count:
    print(f"Data quality check failed for loaded table has data missing while trasfering from source data")
else:
    print(f"Data quality check passed for loaded table has same number of rows as source data")
        


#### 4.3 Data dictionary 
The following the a summary of data dictionary of the data model

#### Fact table
| Feature                | Description                                                                     |
|------------------------|---------------------------------------------------------------------------------|
| record_id              | Unique record ID                                                                |
| country_residence_code | 3 digit code for immigrant country of residence                                 |
| visa_type_key          | A numerical key that links to the visa_type dimension table                     |
| state_code             | US state of arrival                                                             |
| i94yr                  | 4 digit year                                                                    |
| i94mon                 | Numeric month                                                                   |
| i94port                | Port of admission                                                               |
| arrdate                | Arrival Date in the USA                                                         |
| i94mode                | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not   reported)         |
| i94addr                | USA State of arrival                                                            |
| depdate                | Departure Date from the USA                                                     |
| i94bir                 | Age of Respondent in Years                                                      |
| i94visa                | Visa codes collapsed into three categories                                      |
| count                  | Field used for summary statistics                                               |
| dtadfile               | Character Date Field - Date added to I-94 Files                                 |
| visapost               | Department of State where where Visa was issued                                 |
| occup                  | Occupation that will be performed in U.S                                        |
| entdepa                | Arrival Flag - admitted or paroled into the U.S.                                |
| entdepd                | Departure Flag - Departed, lost I-94 or is deceased                             |
| entdepu                | Update Flag - Either apprehended, overstayed, adjusted to perm   residence      |
| matflag                | Match flag - Match of arrival and departure records                             |
| biryear                | 4 digit year of birth                                                           |
| dtaddto                | Character Date Field - Date to which admitted to U.S. (allowed   to stay until) |
| gender                 | Non-immigrant sex                                                               |

#### Country dimension table
| Feature             | Description                    |
|---------------------|--------------------------------|
| country_code        | Unique country code            |
| country_name        | Name of country                |
| average_temperature | Average temperature of country |

#### Visa dimension table
| Feature       | Description                    |
|---------------|--------------------------------|
| visa_type_key | Unique id for each visa issued |
| visa_type     | Name of visa                   |

#### Arrive date dimension table
| Feature         | Description          |
|-----------------|----------------------|
| id              | Unique id            |
| arrdate         | Arrival date into US |
| arrival_year    | Arrival year into US |
| arrival_month   | Arrival MonthS       |
| arrival_day     | Arrival Day          |
| arrival_week    | Arrival Week         |
| arrival_weekday | Arrival WeekDay      |

#### Demographics dimension table
| Feature                | Description                                                   |
|------------------------|---------------------------------------------------------------|
| id                     | Record id                                                     |
| state_code             | US state code                                                 |
| City                   | City Name                                                     |
| State                  | US State where city is located                                |
| Median Age             | Median age of the population                                  |
| Male Population        | Count of male population                                      |
| Female Population      | Count of female population                                    |
| Total Population       | Count of total population                                     |
| Number of Veterans     | Count of total Veterans                                       |
| Foreign born           | Count of residents of the city that were not born in the city |
| Average Household Size | Average city household size                                   |
| Race                   | Respondent race                                               |
| Count                  | Count of city's individual per race                           |

#### Step 5: Complete Project Write Up
* What's the goal? What queries will you want to run? How would Spark or Airflow be incorporated? Why did you choose the model you chose?  
    - This project is going to utilize I94 immigration, global land temperatures and US demographics data to build up analytics database. By combining the datasets appropriately, analysis can be done on the dataset to explore the pattern and insight of US immigration along the time.  

    - The data will load in i94 immigration event, land temperature and US demographic dataset, then clean and process the datasets efficiently with the help of module functions and pipeline, then extract the dimension information from cleaned source data and create the dimension tables. With the fact table which store the immigration events and all dimension tables which store the arrive date, visa type, country averag temperature and demographics data, we can do analysis on:
        - How international traveler distributes within months over the whole year?
        - How international visitor arrival date expand?
        - How people choose which port and which travel mode to enter the US? 
        - How does age and visa status of international visitor look?
        - Is there any relationship between where international visitors come from and the land temperature?  

* The rationale for the choice of tools and technologies for the project.
    - Technology and tools  
The project will store the data on Amazon S3 and use Apache Spark to read in source data from staging tables, extract necessary columns needed for analysis and populate the fact and dimension tables. Then will use Spark to write the data back to S3 if needed. For the data modeling part, the project will use the dimensional model which will make it easy for business users to work with the data and also improve analytical queries performance. So, in this case, we will use Star Schema which fits OLAP (online analytical processing) very well.

    - Why S3 and Spark  
When dealing with the large dataset in this project, with the combining of both batch and streaming capabilities, Spark can support the use case very well where the data need to be stored and analyzed in real-time. It will have more flexibility when more type and volume of data sources need to be added. Therefore, storing the data on S3 will eliminate need to invest in costly hardware and scale up with full flexibility when needed. And speaking of Parquet files, the columnar format that being used will be a good option to store big data set and for analytics purpures as well. And Spark can efficiently read data from S3 and process the data with full sets of data analytics and machine learning libraries. Especially when dealing with large dataset, Spark has more capacity to handle the performance and efficiency.

* How often the data should be updated and why.  
The I94 immigration data used in this project is updated monthly, therefore it would be a good choice to update the data model designed in this project monthly as well.

* More scenarios:
 * The data was increased by 100x.  
     Since Spark is designed for handling big data set, the increased data set won't be a big issue for Spark. But it might be an option to change some setting when setup the clusters like node numbers, computer power, etc.  
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.  
     We can utilize the Apache Airflow to schedule the pipeline running so that we can get time on time everyday.
 * The database needed to be accessed by 100+ people.  
     We can move the database to cloud like using Redshift so that we can support more access better.